In [1]:
import csv
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import datasets

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

In [2]:
device = 'cuda'
model_name = '../models/original_final/'
model_precision = "float32"
max_length = 2048
input_fn = './perturbed_inputs.csv'
output_fn = f'./scores_perturbed:original_final.csv'

In [3]:
tokenizer = AutoTokenizer.from_pretrained('gpt2')
if model_precision == "float16":
    model = AutoModelForCausalLM.from_pretrained(model_name, revision="float16", torch_dtype=torch.float16,
                                                 return_dict=True).to(device)
else:
    model = AutoModelForCausalLM.from_pretrained(model_name, return_dict=True).to(device)

2023-09-01 15:37:54.122157: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [4]:
df = pd.read_csv(input_fn)
df.head(1)

,example_index,text,sub_index,original,synonym
0,920792,I also tried passing the parameters directly t...,276,nice,good


In [5]:
out_fh = open(output_fn, 'wt')
out = csv.writer(out_fh)

In [6]:
for i, row in tqdm(df.iterrows(), total=len(df)):
    line_idx, sentence, char_idx, w1, w2 = row['example_index'], \
                                            row['text'], row['sub_index'], row['original'], row['synonym']
    line_idx, char_idx = int(line_idx), int(char_idx)
    
    # get the first token of each word
    w1_idx = tokenizer.encode(f' {w1}', return_tensors='pt')[0,0].item()
    w2_idx = tokenizer.encode(f' {w2}', return_tensors='pt')[0,0].item()

    input_ids = tokenizer.encode(sentence[:char_idx], \
                                 return_tensors='pt', \
                                 max_length=5000, \
                                 padding=False).to(device)
    input_ids = input_ids[:,-max_length:]
    
    with torch.no_grad():
        model.eval()
        outputs = model(input_ids, labels=input_ids)
        loss = outputs.loss
        logits = outputs.logits

    # Get the loss at each token
    last_logits = logits[..., -1, :].contiguous().squeeze(0)
    probs = torch.nn.Softmax(dim=-1)(last_logits)

    w1_prob = probs[w1_idx].item()
    w2_prob = probs[w2_idx].item()
    w1_rank = (probs > w1_prob).sum().item()
    w2_rank = (probs > w2_prob).sum().item()
    
    if i % 100 == 0:
        print(w1, w2, w1_prob, w2_prob, w1_rank, w2_rank)
        
    out.writerow([line_idx, w1_prob, w2_prob, w1_rank, w2_rank])


  0%|          | 0/22500 [00:00<?, ?it/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


nice good 0.003624517936259508 0.00921230111271143 42 12
nice good 0.00021479644055943936 0.006020482629537582 327 17
nice good 0.01341481227427721 0.038017064332962036 4 3
nice good 0.00015751799219287932 0.0014206800842657685 527 72
nice good 8.013917977223173e-06 0.00020805907843168825 3637 301
size proportion 0.0019398164004087448 5.6596772992634214e-06 51 4868
size proportion 0.003822929924353957 3.143627327517606e-05 28 1084
size proportion 0.0014781958889216185 9.829530426941346e-06 63 4128
size proportion 0.000267009309027344 4.40292751591187e-06 281 5849
size proportion 0.003343340940773487 3.6561363231157884e-05 39 1737
way direction 0.0007744031609036028 2.1511597878998145e-05 208 4126
way direction 0.03237053006887436 0.0001141825268859975 3 959
way direction 0.003909838851541281 0.0003128134994767606 22 497
way direction 7.686941749796006e-09 1.2317876496581448e-07 10279 2729
way direction 0.023319141939282417 7.96807071310468e-05 1 1685
small little 0.08946972340345383 0.

man person 0.0014553600922226906 0.0019879271276295185 101 71
man person 0.06605775654315948 0.0230596624314785 0 5
man person 0.0018464380409568548 0.004019617568701506 75 21
man person 0.06241020932793617 0.0008024381240829825 1 134
own hold 0.059070877730846405 8.966885798145086e-05 0 1228
own hold 0.007957760244607925 2.4813940399326384e-05 11 2586
own hold 0.0059572323225438595 0.00017432101594749838 9 986
own hold 0.057166408747434616 4.108302528038621e-05 0 2481
own hold 0.00017359699995722622 0.0008972046780399978 508 161
choose pick 0.00012397380487527698 6.0101454437244684e-05 361 585
choose pick 4.0000364265324606e-07 6.228395704965806e-06 18097 4521
choose pick 0.001344335381872952 0.002904423512518406 109 52
choose pick 0.0003058865258935839 4.0726336010266095e-05 145 556
choose pick 0.005344139412045479 0.0012845371384173632 33 106
house home 0.0008203687029890716 0.002435382455587387 136 43
house home 1.060458816937171e-05 6.595464219572023e-05 6112 1249
house home 0.003

In [7]:
out_fh.close()